In [1]:
import src.paths

In [2]:
date = '2022-09-16'

In [3]:
%%time
spikes_raw = pd.read_pickle(src.paths.DATA / date / f'{date}-hpc-spikes.pkl')

CPU times: user 300 µs, sys: 633 ms, total: 633 ms
Wall time: 17.9 s


In [4]:
%%time
csd_raw = pd.read_pickle(src.paths.DATA / date / f'{date}-hpc-csd.pkl')

CPU times: user 62.9 ms, sys: 2.71 s, total: 2.77 s
Wall time: 1min 37s


In [6]:
spikes = spikes_raw.loc[start:end - 1 / 250, 160:163]
csd = csd_raw.loc[start:end - 1 / 1250, 40]

In [5]:
start = 300 # start time in seconds
end = 1500  # end time in seconds
Q = 4
w0 = 8

In [ ]:
x = csd.values * 0.001
b, a = scipy.signal.iirnotch(w0, Q, fs=1250)
y = x - scipy.signal.filtfilt(b, a, x, axis=0)
l.append(y)